# ESO 137-001 Flux Comparisons

This notebook compares the flux results from SExtractor to those by PhotUtils.

## Imports

In [ ]:
# Python Imports
from pathlib import Path
from warnings import catch_warnings, simplefilter

In [ ]:
# Util Imports
from tqdm.notebook import tqdm

In [ ]:
# Numerical Imports
import numpy as np

In [ ]:
# Astropy Collaboration Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from astropy.stats import SigmaClip
from astropy.coordinates import SkyCoord
from astropy.convolution import Kernel2D, Gaussian2DKernel, convolve, convolve_fft
from photutils import background
from photutils.utils import calc_total_error
from photutils.segmentation import detect_sources, deblend_sources, SourceCatalog

In [ ]:
# Plotting Imports
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, CustomJS, Whisker
output_notebook()

## Notebook Setup

In [ ]:
# Directories
# Today, I feel like using absolute paths.
CAT_DIR = Path("Catalogs").resolve()
IMG_DIR = Path("../../Images/ProcessedImages/HST/Drizzled").resolve()

# Filters
FILTERS = (275, 475, 814, 160)

## Classes

This is to auto-run SExtractor on the Images. This may be a bad idea...

In [ ]:
# Image Data
# A container for relevant image data
class ImageData:

    # Constructor
    def __init__(
            self, fileName: str,
            sigmaClip: SigmaClip = SigmaClip(sigma=3.0, maxiters=10),
            kernel: Kernel2D = Gaussian2DKernel(3.5*5/3),  # FWHM = 3.5 FLC pix
            bg_box_size: int = 64,
            detection_threshold: float = 3.0,
            no_fft: bool = False,
            drop_nan: bool = True,
            imgUnit: u.Unit = u.electron / u.s
        ):

        # Store the File Name
        self.fileName = fileName

        # Store PhotUtils Parameters
        self._sigmaClip = sigmaClip
        self._kernel = kernel
        self._bg_box_size = bg_box_size
        self._detection_threshold = detection_threshold
        self.no_fft = no_fft
        self.drop_nan = drop_nan

        # Open the FITS File & Get Data
        with fits.open(fileName) as hduList:

            # Get the Primary Header
            self.primaryHeader = hduList[0].header

            # Get the Image Data
            self.imageHeader = hduList[1].header
            self.imageData = hduList[1].data << imgUnit
            self.imageWCS = WCS(self.imageHeader)

            # Get the Weight Data
            # We previously created weights as IVM in AstroDrizzle
            self.weightData = hduList[2].data << (imgUnit**-2)
            self.errorData = 1 / np.sqrt(self.weightData)

        # Get the Background
        self._background = self._get_background()

        # Get the Convolved Image
        self._convolvedData = self._convolve()

        # Get the Sources
        self._segmentImage = self._get_sources()

        # Get the Total Error
        self._totalError = self._calculate_total_error()

        # Get the Catalog
        self.catalog = self._get_catalog()


    # --- Helper Methods --- #
    # Background Subtraction
    def _get_background(self):
        return background.Background2D(
            self.imageData, self._bg_box_size,
            filter_size=3, sigma_clip=self._sigmaClip,
            bkg_estimator=background.SExtractorBackground(sigma_clip=self._sigmaClip)
        )

    # Convolution
    def _convolve(self):
        if np.any(np.array(self._kernel.shape) > 15) and not self.no_fft:
            return convolve_fft(self.residualImage, self._kernel, allow_huge=True)
        else:
            return convolve(self.residualImage, self._kernel)

    # Get Sources
    def _get_sources(self):

        # Get Original Source Map
        segMap = detect_sources(
            self._convolvedData,
            self._detection_threshold * self._background.background_rms,
            npixels=5
        )

        # Use Parallel Deblending?
        nproc = None if segMap.nlabels >= 1000 else 1

        # Deblend and Return
        return deblend_sources(
            self._convolvedData, segMap, npixels=5, nlevels=64,
            nproc=nproc, progress_bar=False
        )

    # Calculate Total Error
    def _calculate_total_error(self):

        # RMS Error (Non-Poisson)
        rms = np.sqrt(self.errorData**2 + self._background.background_rms**2)

        # Return Total Error
        return calc_total_error(
            self.imageData, rms, effective_gain=self.primaryHeader['EXPTIME']*u.s
        )

    # Get Catalog
    def _get_catalog(self):

        # Get Catalog
        cat = SourceCatalog(
            self.imageData, self._segmentImage,
            convolved_data=self._convolvedData,
            error=self._totalError,
            background=self._background.background,
            wcs=self.imageWCS
        )

        # Drop NaN if Needed
        if self.drop_nan:
            cat = cat[~np.isnan(cat.segment_flux)]
            self._segmentImage.keep_labels(cat.labels)

        return cat


    # --- Public Methods --- #
    # Save to DS9
    def to_ds9(self, fileName: str, color: str = 'cyan', ellipse: bool = True):

        # Open the File
        with open(fileName, 'w') as fid:

            # Write the Header
            fid.write('# Region file format: DS9 version 4.1\n')
            fid.write(f'global color={color}\n')
            fid.write(f'{self.imageWCS.wcs.radesys}\n')

            if ellipse:

                # Write the Kron Apertures
                for src in self.catalog:

                    # Get Kron
                    kron = src.kron_aperture.to_sky(self.imageWCS)
                    ra   = kron.positions.ra.value
                    dec  = kron.positions.dec.value
                    a    = kron.a.to(u.arcsec).value
                    b    = kron.b.to(u.arcsec).value
                    theta= kron.theta.to(u.deg).value + 90
                    # Have to add 90 degrees to the theta to match DS9
                    # PhotUtils uses the angle from the x-axis, DS9 uses the angle from North

                    # Write the Ellipse
                    fid.write(f'ellipse({ra},{dec},{a}",{b}",{theta}) # text={{{src.label}}}\n')

            else:

                # Write the Sources
                for src in self.catalog:
                    ra  = src.sky_centroid.ra.value
                    dec = src.sky_centroid.dec.value
                    fid.write(f'point({ra},{dec}) # text={{{src.label}}}\n')

    # Use Other for Detection
    def use_other(self, other: 'ImageData'):

        return SourceCatalog(
            self.imageData, other.segmentImage,
            convolved_data=self._convolvedData,
            error=self._totalError,
            background=self._background.background,
            wcs=self.imageWCS, detection_cat=other.catalog
        )

    # -- Static Methods --- #
    @staticmethod
    def calculate_magnitude(flux: u.Quantity, zp: u.Quantity) -> u.Quantity:
        """Calculate the magnitude from the flux and zero point.

        Inputs
        ------
        flux : `~astropy.units.Quantity`
            The flux of the source in any unit that is not logarithmic.
        zp : `~astropy.units.Quantity`
            The zero point for the image in logarithmic units.
        """
        return u.Magnitude(flux) + zp

    # --- Property Methods --- #
    @property
    def background(self):
        return self._background

    @property
    def residualImage(self):
        return self.imageData - self._background.background

    @property
    def segmentImage(self):
        return self._segmentImage

    @property
    def zpST(self):
        zp = -2.5 * np.log10(self.imageHeader['PHOTFLAM']) - 21.1
        return u.Magnitude(zp, unit=u.mag(u.ST/self.imageData.unit))

    @property
    def zpAB(self):
        zp  = self.zpST.value
        zp += 18.6921
        zp -= 5 * np.log10(self.imageHeader['PHOTPLAM'])
        return u.Magnitude(zp, unit=u.mag(u.AB/self.imageData.unit))

    @property
    def pixelScale(self):
        return self.imageWCS.proj_plane_pixel_scales()[0].to(u.arcsec) / u.pix

    # --- Setter/Getter Methods --- #
    @property
    def sigmaClip(self):
        return self._sigmaClip

    @sigmaClip.setter
    def sigmaClip(self, value: SigmaClip):
        self._sigmaClip = value
        self._background = self._get_background()
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def kernel(self):
        return self._kernel

    @kernel.setter
    def kernel(self, value: Kernel2D):
        self._kernel = value
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def bg_box_size(self):
        return self._bg_box_size

    @bg_box_size.setter
    def bg_box_size(self, value: int):
        self._bg_box_size = value
        self._background = self._get_background()
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def detection_threshold(self):
        return self._detection_threshold

    @detection_threshold.setter
    def detection_threshold(self, value: float):
        self._detection_threshold = value
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

## Load Data

### Load the SExtractor Data

We will just be looking at the catalogs where the detection/measurement images are the same.

In [ ]:
# Loop through filters and load SExtractor catalogs
seCats = {}

for filt in FILTERS:
    catName = CAT_DIR / f"ESO_F{filt}WxF{filt}W.cat"
    seCats[filt] = QTable.read(catName, format='ascii.sextractor')
    seCats[filt]['sky_centroid'] = SkyCoord(
        ra=seCats[filt]['X_WORLD'],
        dec=seCats[filt]['Y_WORLD'],
        frame='fk5'
    )

### Calculate the PhotUtils Data

In [ ]:
# Set the Convolution Pixel Size
# I don't think I will use this going forward, but these values are here
# to match SExtractor
CONV_PIXELS = {
    275: 2.5,
    475: 3.0,
    814: 3.0,
    160: 5.0
}

# Aperture Sizes
APER_RADII = {
    275: 18.333,
    475: 16.667,
    814: 16.667,
    160: 26.667
}


In [ ]:
# Load the Image Data for Each Image
photUtilCats = {}
for filt in tqdm(FILTERS):

    # Set the CTE Character
    cteChar = 'c' if filt != 160 else 'z'

    # Set the File Name
    imgName = IMG_DIR / f"ESO137-001-F{filt}W_dr{cteChar}.fits"

    # Set the Image Data Parameters
    dataParams = {
        'sigmaClip': SigmaClip(sigma=3, maxiters=10),
        'kernel': Gaussian2DKernel(CONV_PIXELS[filt]),  # FWHM = 3.5 FLC pix
        'bg_box_size': 128,
        'detection_threshold': 1.5,
        'no_fft': True
    }

    with catch_warnings():
        simplefilter("ignore")
        cat = ImageData(imgName, **dataParams).catalog
        _ = cat.circular_photometry(APER_RADII[filt], name='aper')
        photUtilCats[filt] = cat.to_table(
            ['label', 'sky_centroid', 'aper_flux', 'aper_fluxerr']
        )

## Compare Results

Make a plot for each of the four filters to compare the measudarkorange fluxes.

In [ ]:
# Set the Max Sep
MAX_SEP = 0.2 * u.arcsec

### F275W Comparison

In [ ]:
# Get the Matching PhotUtils Sources to the SExtractor Sources
idx, sep, _ = photUtilCats[275]['sky_centroid'].match_to_catalog_sky(seCats[275]['sky_centroid'])
goodMatch = sep < MAX_SEP
goodMatch &= np.isfinite(photUtilCats[275]['aper_fluxerr'])  # Ensure PhotUtils flux is finite
matchedPUCat = photUtilCats[275][goodMatch]
matchedSECat = seCats[275][idx[goodMatch]]

In [ ]:
# Create a ColumnDataSource for the scatter plot
scatterSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'sky_centroid': matchedPUCat['sky_centroid'].to_string()
})
xErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedPUCat['aper_flux'].value-matchedPUCat['aper_fluxerr'].value,
    'upper': matchedPUCat['aper_flux'].value+matchedPUCat['aper_fluxerr'].value
})
yErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedSECat['FLUX_APER'].value-matchedSECat['FLUXERR_APER'].value,
    'upper': matchedSECat['FLUX_APER'].value+matchedSECat['FLUXERR_APER'].value
})

# Create the figure
p = figure(
    x_axis_type="log", y_axis_type="log",
    x_range=(0.01, 10**3), y_range=(0.01, 10**3),
    width=600, height=600,
    title="F275W Flux Comparison: PhotUtils vs SExtractor",
    tools="pan,box_zoom,reset,save"
)

# Add the y=x line
p.line(
    x=[0.01, 10**3], y=[0.01, 10**3],
    line_dash="dashed", line_color="black", line_width=2
)

# Add the flux points with error bars
p.scatter(
    x='x', y='y', source=scatterSrc,
    size=5, color="darkorange", legend_label="Flux Points"
)
xErr = Whisker(
    source=xErrSrc, base="y", upper="upper", lower="lower",
    dimension="width", line_color="gray"
)
xErr.upper_head.size = xErr.lower_head.size = 5
p.add_layout(xErr)
yErr = Whisker(
    source=yErrSrc, base="x", upper="upper", lower="lower",
    dimension="height", line_color="gray"
)
yErr.upper_head.size = yErr.lower_head.size = 5
p.add_layout(yErr)

# Configure hover tool
hover = HoverTool()
hover.tooltips = [
    ("PhotUtils Flux", "@x"),
    ("SExtractor Flux", "@y"),
    ("Sky Coord", "@sky_centroid")
]
hover.mode = "mouse"
p.add_tools(hover)

# Add axis labels
p.xaxis.axis_label = "PhotUtils Flux (electron/s)"
p.yaxis.axis_label = "SExtractor Flux (electron/s)"

# Show the plot
show(p)

### F475W Comparison

In [ ]:
# Get the Matching PhotUtils Sources to the SExtractor Sources
idx, sep, _ = photUtilCats[475]['sky_centroid'].match_to_catalog_sky(seCats[475]['sky_centroid'])
goodMatch = sep < MAX_SEP
goodMatch &= np.isfinite(photUtilCats[475]['aper_fluxerr'])  # Ensure PhotUtils flux is finite
matchedPUCat = photUtilCats[475][goodMatch]
matchedSECat = seCats[475][idx[goodMatch]]

In [ ]:
# Create a ColumnDataSource for the scatter plot
scatterSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'sky_centroid': matchedPUCat['sky_centroid'].to_string()
})
xErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedPUCat['aper_flux'].value-matchedPUCat['aper_fluxerr'].value,
    'upper': matchedPUCat['aper_flux'].value+matchedPUCat['aper_fluxerr'].value
})
yErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedSECat['FLUX_APER'].value-matchedSECat['FLUXERR_APER'].value,
    'upper': matchedSECat['FLUX_APER'].value+matchedSECat['FLUXERR_APER'].value
})

# Create the figure
p = figure(
    x_axis_type="log", y_axis_type="log",
    x_range=(0.01, 10**4), y_range=(0.01, 10**4),
    width=600, height=600,
    title="F475W Flux Comparison: PhotUtils vs SExtractor",
    tools="pan,box_zoom,reset,save"
)

# Add the y=x line
p.line(
    x=[0.01, 10**4], y=[0.01, 10**4],
    line_dash="dashed", line_color="black", line_width=2
)

# Add the flux points with error bars
p.scatter(
    x='x', y='y', source=scatterSrc,
    size=5, color="darkorange", legend_label="Flux Points"
)
xErr = Whisker(
    source=xErrSrc, base="y", upper="upper", lower="lower",
    dimension="width", line_color="gray"
)
xErr.upper_head.size = xErr.lower_head.size = 5
p.add_layout(xErr)
yErr = Whisker(
    source=yErrSrc, base="x", upper="upper", lower="lower",
    dimension="height", line_color="gray"
)
yErr.upper_head.size = yErr.lower_head.size = 5
p.add_layout(yErr)

# Configure hover tool
hover = HoverTool()
hover.tooltips = [
    ("PhotUtils Flux", "@x"),
    ("SExtractor Flux", "@y"),
    ("Sky Coord", "@sky_centroid")
]
hover.mode = "mouse"
p.add_tools(hover)

# Show the plot
show(p)

### F814W Comparison

In [ ]:
# Get the Matching PhotUtils Sources to the SExtractor Sources
idx, sep, _ = photUtilCats[814]['sky_centroid'].match_to_catalog_sky(seCats[814]['sky_centroid'])
goodMatch = sep < MAX_SEP
goodMatch &= np.isfinite(photUtilCats[814]['aper_fluxerr'])  # Ensure PhotUtils flux is finite
matchedPUCat = photUtilCats[814][goodMatch]
matchedSECat = seCats[814][idx[goodMatch]]

In [ ]:
# Create a ColumnDataSource for the scatter plot
scatterSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'sky_centroid': matchedPUCat['sky_centroid'].to_string()
})
xErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedPUCat['aper_flux'].value-matchedPUCat['aper_fluxerr'].value,
    'upper': matchedPUCat['aper_flux'].value+matchedPUCat['aper_fluxerr'].value
})
yErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedSECat['FLUX_APER'].value-matchedSECat['FLUXERR_APER'].value,
    'upper': matchedSECat['FLUX_APER'].value+matchedSECat['FLUXERR_APER'].value
})

# Create the figure
p = figure(
    x_axis_type="log", y_axis_type="log",
    x_range=(10**-1, 10**4), y_range=(10**-1, 10**4),
    width=600, height=600,
    title="F814W Flux Comparison: PhotUtils vs SExtractor",
    tools="pan,box_zoom,reset,save"
)

# Add the y=x line
p.line(
    x=[10**-1, 10**4], y=[10**-1, 10**4],
    line_dash="dashed", line_color="black", line_width=2
)

# Add the flux points with error bars
p.scatter(
    x='x', y='y', source=scatterSrc,
    size=5, color="darkorange", legend_label="Flux Points"
)
xErr = Whisker(
    source=xErrSrc, base="y", upper="upper", lower="lower",
    dimension="width", line_color="gray"
)
xErr.upper_head.size = xErr.lower_head.size = 5
p.add_layout(xErr)
yErr = Whisker(
    source=yErrSrc, base="x", upper="upper", lower="lower",
    dimension="height", line_color="gray"
)
yErr.upper_head.size = yErr.lower_head.size = 5
p.add_layout(yErr)

# Configure hover tool
hover = HoverTool()
hover.tooltips = [
    ("PhotUtils Flux", "@x"),
    ("SExtractor Flux", "@y"),
    ("Sky Coord", "@sky_centroid")
]
hover.mode = "mouse"
p.add_tools(hover)

# Add axis labels
p.xaxis.axis_label = "PhotUtils Flux (electron/s)"
p.yaxis.axis_label = "SExtractor Flux (electron/s)"

# Show the plot
show(p)

### F160W Comparison

In [ ]:
# Get the Matching PhotUtils Sources to the SExtractor Sources
idx, sep, _ = photUtilCats[160]['sky_centroid'].match_to_catalog_sky(seCats[160]['sky_centroid'])
goodMatch = sep < MAX_SEP
goodMatch &= np.isfinite(photUtilCats[160]['aper_fluxerr'])  # Ensure PhotUtils flux is finite
matchedPUCat = photUtilCats[160][goodMatch]
matchedSECat = seCats[160][idx[goodMatch]]

In [ ]:
# Create a ColumnDataSource for the scatter plot
scatterSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'sky_centroid': matchedPUCat['sky_centroid'].to_string()
})
xErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedPUCat['aper_flux'].value-matchedPUCat['aper_fluxerr'].value,
    'upper': matchedPUCat['aper_flux'].value+matchedPUCat['aper_fluxerr'].value
})
yErrSrc = ColumnDataSource(data={
    'x': matchedPUCat['aper_flux'].value,
    'y': matchedSECat['FLUX_APER'].value,
    'lower': matchedSECat['FLUX_APER'].value-matchedSECat['FLUXERR_APER'].value,
    'upper': matchedSECat['FLUX_APER'].value+matchedSECat['FLUXERR_APER'].value
})

# Create the figure
p = figure(
    x_axis_type="log", y_axis_type="log",
    x_range=(0.01, 10**4), y_range=(0.01, 10**4),
    width=600, height=600,
    title="F160W Flux Comparison: PhotUtils vs SExtractor",
    tools="pan,box_zoom,reset,save"
)

# Add the y=x line
p.line(
    x=[0.01, 10**4], y=[0.01, 10**4],
    line_dash="dashed", line_color="black", line_width=2
)

# Add the flux points with error bars
p.scatter(
    x='x', y='y', source=scatterSrc,
    size=5, color="darkorange", legend_label="Flux Points"
)
xErr = Whisker(
    source=xErrSrc, base="y", upper="upper", lower="lower",
    dimension="width", line_color="gray"
)
xErr.upper_head.size = xErr.lower_head.size = 5
p.add_layout(xErr)
yErr = Whisker(
    source=yErrSrc, base="x", upper="upper", lower="lower",
    dimension="height", line_color="gray"
)
yErr.upper_head.size = yErr.lower_head.size = 5
p.add_layout(yErr)

# Configure hover tool
hover = HoverTool()
hover.tooltips = [
    ("PhotUtils Flux", "@x"),
    ("SExtractor Flux", "@y"),
    ("Sky Coord", "@sky_centroid")
]
hover.mode = "mouse"
p.add_tools(hover)

# Show the plot
show(p)